# The QR Algorithm

First, we will import the required packages


In [1]:
!pip install numpy

In [0]:
import numpy as np

We are going to create a few utility methods here for future use.

In [0]:
def show(name, arr):
    print(name + ':', arr)

First we must code QR decomposition. For this, we start with a real square matrix $A$ and express it in the form $A = QR$, where $Q$ is an orthogonal matrix and $R$ is an upper-triangular matrix.

Consider our $N \times N$ matrix as a set of $N$ columnb vectors $\vec{a_0}...\vec{a_{N-1}}$

$$A = \begin{pmatrix} 
  \vec{a_0}, \vec{a_1}, ...\vec{a_{N-1}}
\end{pmatrix}$$

Now we define two new sets of vectors $\vec{u_0}...\vec{u_{N-1}}$ and $\vec{q_0}...\vec{q_{N-1}}$ as follows:

$$\vec{u_0} = \vec{a_0} \qquad \vec{q_0} = \dfrac{\vec{u_0}}{||\vec{u_0}||}$$
$$\vec{u_1} = \vec{a_1} - (\vec{q_0} \cdot \vec{a_1})\vec{q_0} \qquad \vec{q_1} = \dfrac{\vec{u_1}}{||\vec{u_1}||}$$
$$\vec{u_2} = \vec{a_2} - (\vec{q_0} \cdot \vec{a_2})\vec{q_0} - (\vec{q_1} \cdot \vec{a_2})\vec{q_1} \qquad \vec{q_2} = \dfrac{\vec{u_2}}{||\vec{u_2}||}$$
Or in general,
$$\vec{u_i} = \vec{a_i} - \sum_{j = 0}^{i - 1}(\vec{q_j} \cdot \vec{a_i})\vec{q_j} \qquad \vec{q_i} = \dfrac{\vec{u_i}}{||\vec{u_i}||}$$
This is called the Gram Schmidt Orthogonalization Process, where the set of $\vec{u_i}$ vectors are orthogonal and the set of $\vec{q_i}$ vectors are orthonormal.

This can be shown by taking the dot product,
$$q_i \cdot q_j = \left\{
\begin{array}{ll}
      1 & \text{ if } i = j \\
      0 & \text{ if } i \neq j
\end{array} 
\right. $$

Now, rearranging, we have
$$\vec{a_0} = ||\vec{u_0}||\vec{q_0}$$
$$\vec{a_1} = ||\vec{u_1}||\vec{q_1} + (\vec{q_0} \cdot \vec{a_1})\vec{q_0}$$
$$\vec{a_2} = ||\vec{u_2}||\vec{q_2} + (\vec{q_0} \cdot \vec{a_2})\vec{q_0} + (\vec{q_1} \cdot \vec{a_2})\vec{q_1}$$
Or in general,
$$\vec{a_i} = ||\vec{u_i}||\vec{q_i} + \sum_{j = 0}^{i - 1}(\vec{q_j} \cdot \vec{a_i})\vec{q_j}$$

Now, we can group together the vectors $\vec{q_i}$ as the columns of a matrix and write all these equations as 
$$
A = \begin{pmatrix}
  \vec{a_0} & \vec{a_1} & \vec{a_2} & \dots 
\end{pmatrix} = \begin{pmatrix}
  \vec{q_0} & \vec{q_1} & \vec{q_2} & \dots 
\end{pmatrix} \begin{pmatrix}
  ||\vec{u_0}|| & q_0 \cdot a_1 & q_0 \cdot a_2 & \dots \\
  0 & ||\vec{u_1}|| & q_1 \cdot a_2 & \dots \\
  0 & 0 & ||\vec{u_3}|| & \dots \\
  \vdots & \vdots & \vdots & \ddots
\end{pmatrix}
$$

These two matricies satify our needs as $Q$ and $R$,
$$Q = \begin{pmatrix}
  \vec{q_0} & \vec{q_1} & \vec{q_2} & \dots 
\end{pmatrix} \qquad R = \begin{pmatrix}
  ||\vec{u_0}|| & q_0 \cdot a_1 & q_0 \cdot a_2 & \dots \\
  0 & ||\vec{u_1}|| & q_1 \cdot a_2 & \dots \\
  0 & 0 & ||\vec{u_3}|| & \dots \\
  \vdots & \vdots & \vdots & \ddots
\end{pmatrix}$$

Now, to write this in code as a function:



In [0]:
def qr_decomposition(A):
    N = len(A)

    U = np.empty([N, N])
    Q = np.empty([N, N])

    for i in range(N):
        U[:, i] = A[:, i] - sum([(np.dot(Q[:, j], A[:, i])) * Q[:, j] for j in range(i)])
        Q[:, i] = U[:, i] / np.linalg.norm(U[:, i])

    R = np.empty([N, N])

    for i in range(N):
        for j in range(0, N):
            R[i, j] = np.dot(Q[:, i], A[:, j]) if j > i else 0
        R[i, i] = np.linalg.norm(U[:, i])

    return Q, R

Create a test matrix A

In [0]:
A = np.array([[1, 4, 8, 4], [4, 2, 3, 7], [8, 3, 6, 9], [4, 7, 9, 2]])

Now to test this function on the matrix A

In [6]:
Q, R = qr_decomposition(A)

show("A", A)
show("Q", Q)
show("R", R)
show("Q * R", np.matmul(Q, R))


A: [[1 4 8 4]
 [4 2 3 7]
 [8 3 6 9]
 [4 7 9 2]]
Q: [[ 0.10153462  0.558463    0.80981107  0.1483773 ]
 [ 0.40613847 -0.10686638 -0.14147555  0.8964462 ]
 [ 0.81227693 -0.38092692  0.22995024 -0.37712564]
 [ 0.40613847  0.72910447 -0.5208777  -0.17928924]]
R: [[ 9.8488578   6.49821546 10.55960012 11.37187705]
 [ 0.          5.98106979  8.4234836  -0.484346  ]
 [ 0.          0.          2.74586406  3.27671222]
 [ 0.          0.          0.          3.11592335]]
Q * R: [[1. 4. 8. 4.]
 [4. 2. 3. 7.]
 [8. 3. 6. 9.]
 [4. 7. 9. 2.]]


Now, using this function, we can implement the QR algorithm as follows:

1.   Create an $N \times N$ matrix $V$ to hold the eigenvectors and initially set it equal to the identity matrx $I$. Also choose a target accuracy $\epsilon$ for the off-diagnonal elements of the eigenvectors.
2.   Calculate the QR decomposition $A = QR$
3.   Update $A$ to the new value $A = RQ$
4.   Multiply $V$ on the right by $Q$
5.   Check the off-diagonal elements of $A$. If they are all less than $\epsilon$, we are done. Otherwise go back to step 2.

Now, we code this into a function to find the eigenvalues and eigenvectors of a matrix $A$.



In [0]:
def eigh(A, eps=0.000001, max_iterations=1000):

    N = len(A)
    V = np.identity(N)

    c = 0

    while True:

        c += 1

        Q, R = qr_decomposition(A)
        A = np.matmul(R, Q)
        V = np.matmul(V, Q)

        done = True

        for i in range(N):
            if not done: break
            for j in range(N):
                if A[i, j] > eps and i != j:
                    done = False
                    break

        if c >= max_iterations:
            done = True

        if done:
            x = np.array([A[i, i] for i in range(N)])
            
            return x, V


Now to test this function on a matrix A

In [8]:
x, V = eigh(A)
show("x", x)
show("V", V)

x: [21. -8. -3.  1.]
V: [[ 0.43151698 -0.38357064 -0.77459666 -0.25819889]
 [ 0.38357063  0.43151698 -0.2581989   0.77459667]
 [ 0.62330228  0.52740965  0.25819889 -0.51639778]
 [ 0.52740965 -0.62330227  0.51639779  0.25819889]]
